In [ ]:
/* ML.NET Model Builder generated Notebook file. Notebook files contain both code snippets and rich text elements.
Use the "run" button in the left margin to execute each code snippet and explore ML.NET. */

#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json" 
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json"
#i "nuget:https://mlnetcli.blob.core.windows.net/mlnetcli/index.json"

// add nightly build for ml.net
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"

Restore sources https://mlnetcli.blob.core.windows.net/mlnetcli/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json

In [ ]:
#r "nuget:MLNetAutoML.InteractiveExtension,0.1.1"
#r "nuget:Microsoft.ML.AutoML,0.20.0-preview.22226.2"
#r "nuget:Microsoft.ML.TimeSeries,2.0.0-preview.22226.2"
#r "nuget:Microsoft.Data.Analysis,0.20.0-preview.22226.2"
#r "nuget:XPlot.Plotly.Interactive,4.0.6"

Restore sources https://mlnetcli.blob.core.windows.net/mlnetcli/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22226.2 Microsoft.ML.AutoML, 0.20.0-preview.22226.2 Microsoft.ML.TimeSeries, 2.0.0-preview.22226.2 MLNetAutoML.InteractiveExtension, 0.1.1 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `MLNetAutoML.InteractiveExtension.dll`

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [ ]:
// Import common usings.
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.Data.Analysis;
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using Microsoft.ML.SearchSpace;
using Microsoft.ML.SearchSpace.Tuner;
using Microsoft.ML.Transforms.Text;
using Microsoft.ML.AutoML;
using System;
using System.Collections.Generic;
using System.Text;
using System.Text.Json;
using Microsoft.ML.Runtime;
using MLNetAutoML.InteractiveExtension;
using System.Diagnostics;
using Microsoft.ML.Transforms.TimeSeries;

In [ ]:

public class ForecastInput
{
    [ColumnName("load")]
    public float Load { get; set; }
}

public class ForecastOutnput
{
    [ColumnName("predict")]
    public float[] Predict { get; set; }
}

public class CustomRunner : ITrialRunner
{
    private MLContext _context;
    private IDataView _train;
    private IDataView _test;

    public CustomRunner(MLContext context, IDataView train, IDataView test)
    {
        this._context = context;
        this._train = train;
        this._test = test;
    }

    public TrialResult Run(TrialSettings settings, IServiceProvider provider)
    {
        try
            {
                var trainDataset = this._train;
                var testDataset = this._test;

                var stopWatch = new Stopwatch();
                stopWatch.Start();
                var pipeline = settings.Pipeline.BuildTrainingPipeline(this._context, settings.Parameter);
                var model = pipeline.Fit(trainDataset);
                var predictEngine = model.CreateTimeSeriesEngine<ForecastInput, ForecastOutnput>(this._context);

                var predictedLoad1H = new List<float>();
                var predictedLoad2H = new List<float>();
                var N = testDataset.GetRowCount();

                // evaluate
                foreach (var load in testDataset.GetColumn<Single>("load"))
                {
                    // firstly, get next n predict where n is horizon
                    var predict = predictEngine.Predict();

                    predictedLoad1H.Add(predict.Predict[0]);
                    // predictedLoad2H.Add(predict.Predict[1]);

                    // update model with truth value
                    predictEngine.Predict(new ForecastInput()
                    {
                        Load = load,
                    });
                }

                var rmse1H = Enumerable.Zip(testDataset.GetColumn<float>("load"), predictedLoad1H)
                                       .Select(x => Math.Pow(x.First - x.Second, 2))
                                       .Average();
                rmse1H = Math.Sqrt(rmse1H);
                stopWatch.Stop();
                var rmse = rmse1H;

                return new TrialResult()
                {
                    Metric = rmse,
                    Model = model,
                    TrialSettings = settings,
                    DurationInMilliseconds = stopWatch.ElapsedMilliseconds,
                };

            }
            catch (Exception)
            {
                return new TrialResult()
                {
                    Metric = double.MaxValue,
                    Model = null,
                    TrialSettings = settings,
                    DurationInMilliseconds = 0,
                };
            }

        throw new ArgumentException();
    }
}
public class ForecastBySsaSearchSpace
{
    [Range(2, 24 * 7 * 30)]
    public int WindowSize { get; set; } = 24 *3;

    [Range(2, 24 * 7 * 30)]
    public int SeriesLength { get; set; } = 24 * 7;

    [Range(1, 24 * 7 * 30)]
    public int Rank { get; set; } = 1;

    [Range(24 * 7, 3975)]
    public int TrainSize { get; set; } = 3975;

    [Range(1,2)]
    public int Horizon { get; set; } = 1;

    [Range(0.00001F,1F)]
    public float DiscountFactor {get; set;} = 1;
}

In [ ]:
// Load file
var trainDataPath = Path.Combine(Directory.GetCurrentDirectory(),"../Data", "Sonar", "-0401_load.csv");
var evaluatePath = Path.Combine(Directory.GetCurrentDirectory(),"../Data", "Sonar", "0401_0415_load.csv");;
var predictedPath =Path.Combine(Directory.GetCurrentDirectory(),"../Data", "Sonar", "predicted.csv");

var trainDf = DataFrame.LoadCsv(trainDataPath);
var evaluateDf = DataFrame.LoadCsv(evaluatePath);

var mlContext = new MLContext();
var searchSpace = new SearchSpace<ForecastBySsaSearchSpace>();
var runner = new CustomRunner(mlContext, trainDf, evaluateDf);
var monitor = new NotebookMonitor();

Console.WriteLine($"train data Length: {trainDf.Rows.Count}");
var pipeline = mlContext.Transforms.CopyColumns("newLoad", "load")
    .Append(mlContext.Auto().CreateSweepableEstimator((context, ss) =>
    {
        return mlContext.Forecasting.ForecastBySsa("predict", "load", ss.WindowSize, ss.SeriesLength, ss.TrainSize, ss.Horizon, rank: ss.Rank, variableHorizon: true, discountFactor: ss.DiscountFactor);
    }, searchSpace));

var autoMLExperiment = mlContext.Auto().CreateExperiment();

autoMLExperiment.SetPipeline(pipeline)
                .SetTrialRunner(runner)
                .SetTrainingTimeInSeconds(60)
                .SetEvaluateMetric(RegressionMetric.RootMeanSquaredError)
                .SetIsMaximizeMetric(false)
                .SetMonitor(monitor);
                
monitor.SetUpdate(monitor.Display());
var res = autoMLExperiment.Run().Result;
var bestModel = res.Model;

// evaluate
var predictEngine = bestModel.CreateTimeSeriesEngine<ForecastInput, ForecastOutnput>(mlContext);

var predictLoads1H = new List<float>();
var predictLoads2H = new List<float>();
predictLoads2H.Add(0);
foreach (var load in evaluateDf.GetColumn<Single>("load"))
{
    // firstly, get next n predict where n is horizon
    var predict = predictEngine.Predict();

    predictLoads1H.Add(predict.Predict[0]);
    // predictLoads2H.Add(predict.Predict[1]);

    // update model with truth value
    predictEngine.Predict(new ForecastInput()
    {
        Load = load,
    });
}

evaluateDf["predict_load_1h"] = DataFrameColumn.Create("predict_load_1h", predictLoads1H);
// evaluateDf["predict_load_2h"] = DataFrameColumn.Create("predict_load_2h", predictLoads2H.SkipLast(1));
DataFrame.WriteCsv(evaluateDf, predictedPath);

train data Length: 3975


index,Trial,Metric,Pipeline
0,0,Infinity,Unknown=>Unknown
1,1,Infinity,Unknown=>Unknown
2,2,458224.3,Unknown=>Unknown
3,3,455892.12,Unknown=>Unknown
4,4,Infinity,Unknown=>Unknown
5,5,456228.72,Unknown=>Unknown
6,6,Infinity,Unknown=>Unknown
7,7,458224.3,Unknown=>Unknown
8,8,Infinity,Unknown=>Unknown
9,9,456558.7,Unknown=>Unknown
